<a href="https://colab.research.google.com/github/yamaguchinorimitsu/ajax_app/blob/master/%E7%B7%B4%E7%BF%92%EF%BC%91%EF%BC%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# CSVファイルを読み込み
df = pd.read_csv('/content/Boston.csv')


In [3]:
# 欠損値の確認
missing_values = df.isnull().sum()
print("欠損値の数:\n", missing_values)


欠損値の数:
 CRIME      0
ZN         0
INDUS      0
CHAS       0
NOX        1
RM         0
AGE        0
DIS        0
RAD        1
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64


In [4]:
# 欠損値を平均値で補完
df = df.fillna(df.mean())


<ipython-input-4-aad0a10717fb>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


In [5]:
# 欠損値の確認
missing_values = df.isnull().sum()
print("欠損値の数:\n", missing_values)

欠損値の数:
 CRIME      0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64


In [6]:
df.head(2)

,CRIME,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,high,0.0,18.10,0,0.718,3.561,87.9,1.6132,24.0,666,20.2,354.7,7.12,27.5
1,low,0.0,8.14,0,0.538,5.950,82.0,3.9900,4.0,307,21.0,232.6,27.71,13.2


In [7]:
# ダミー変数への変換
crime = pd.get_dummies(df['CRIME'], prefix='crime', drop_first=True)

# 元のデータフレームとダミー変数を横方向に連結
df2 = pd.concat([df, crime], axis=1)

df2 = df2.drop(['CRIME'], axis = 1)

In [8]:
from sklearn.preprocessing import StandardScaler

# StandardScalerのインスタンスを作成
scaler = StandardScaler()

# 訓練データを標準化
sc_df2 = scaler.fit_transform(df2)


In [23]:
from sklearn.decomposition import PCA


# PCAのインスタンスを作成し、主成分数(2か３が多いが次元の数まで作成できる)
# 白色化の設定を指定
model = PCA(n_components=3, whiten=True)

In [24]:
model.fit(sc_df2)

PCA(n_components=3, whiten=True)

In [11]:
# 第一固有ベクトル
print(model.components_[0])

[-0.2258543   0.35923465  0.04220985  0.3499321  -0.19485285  0.29792086
 -0.29980115  0.30726517  0.32822012  0.16246983 -0.18251937  0.27543839
 -0.2018449   0.03831172 -0.31492126]


In [25]:
# 学習済みの主成分分析モデルで新規データを変換
new = model.transform(sc_df2)  # 新規データを主成分に変換

new_df = pd.DataFrame(new)

new_df.head()

,0,1,2
0,1.490417,-0.680415,0.955620
1,0.703223,-0.252517,-2.097557
2,-1.403756,-0.613175,0.518589
3,1.030971,0.560933,-1.364731
4,0.898674,0.272037,1.726017


In [26]:
new_df.columns = ['PC1','PC2','PC3']

df3 = pd.DataFrame(sc_df2, columns=df2.columns)

df4 = pd.concat([df3,new_df],axis=1)

In [ ]:
df4_corr = df4.corr()

df4_corr.loc[:,'PC1':]

In [37]:
# 降順に並び替え
sorted_train_col = df4_corr['PC2'].sort_values(ascending=False)

# 結果を表示
print(sorted_train_col)

PC2               1.000000e+00
PRICE             6.543207e-01
crime_low         6.228277e-01
RM                5.965525e-01
AGE               2.956687e-01
CHAS              2.917857e-01
NOX               2.036776e-01
B                 8.345601e-02
INDUS             4.180095e-02
PC3              -9.139715e-17
PC1              -5.024040e-16
RAD              -1.514608e-01
TAX              -1.702032e-01
ZN               -2.260973e-01
LSTAT            -2.630427e-01
crime_very_low   -4.085430e-01
DIS              -4.324781e-01
PTRATIO          -5.018551e-01
Name: PC2, dtype: float64


In [39]:
model = PCA(whiten=True)

In [40]:
# 主成分分析を実行
X_pca = model.fit_transform(sc_df2)

# 寄与率を取得
variance_ratio = model.explained_variance_ratio_

# 結果を表示
print("寄与率:", variance_ratio)

寄与率: [0.41102789 0.14484698 0.10192698 0.06448954 0.06233684 0.05810331
 0.04843711 0.02885228 0.02142431 0.01831962 0.01572944 0.01068611
 0.00918466 0.00277548 0.00185945]


In [43]:
# 寄与率の合計を保持する変数
cumulative_ratio = 0.0

# 新規の列の数をカウントする変数
num_columns = 0

# 寄与率のリスト
variance_ratio = model.explained_variance_ratio_

# forループで累積の寄与率を計算
for ratio in variance_ratio:
    cumulative_ratio += ratio
    num_columns += 1
    if cumulative_ratio >= 0.8:
        break

# 結果を表示
print("新規の列の数: n=", num_columns)


新規の列の数: n= 6
